In [13]:
#installing necessary library
%pip install nibabel matplotlib

Note: you may need to restart the kernel to use updated packages.


In [14]:
#setting up helper functions
#imports

import argparse
from pathlib import Path
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt


In [15]:
#file finding function for single file

def find_file(case_dir : Path, modality : str) -> Path:
    found = list(case_dir.glob(f"*_{modality}.nii*"))
    if not found:
        raise FileNotFoundError (f"couldnt find the modality '{modality}' in {dir}")
    return found[0]



In [16]:
def load_case(case_dir : Path, modalities=("flair",), seg_mod="seg"):
    vols = {}
    for m in modalities:
        p = find_file(case_dir, m)
        vols[m] =  nib.load(str(p)).get_fdata().astype(np.float32)
        seg_p = find_file(case_dir, seg_mod)
        seg = nib.load(str(seg_p)).get_fdata().astype(np.int16)
        wt = (seg > 0).astype(np.uint8)
        return vols, wt
    

In [17]:
def pick_slices(wt: np.ndarray, min_pos: int = 200):
    z_list = []
    for z in range(wt.shape[2]):
        cnt = int((wt[:, :, z] > 0).sum())
        if cnt >= min_pos:
            z_list.append((z, cnt))
    return z_list

In [18]:
def save_slice_and_overlay(flair: np.ndarray, wt: np.ndarray, z: int, out_dir: Path, prefix: str):
    out_dir.mkdir(parents=True, exist_ok=True)
    sl_img = flair[:, :, z]
    sl_mask = wt[:, :, z].astype(bool)

    # Save raw FLAIR
    plt.figure(figsize=(4,4))
    plt.imshow(sl_img, cmap="gray")
    plt.axis("off")
    plt.title(f"FLAIR z={z}")
    plt.tight_layout()
    f1 = out_dir / f"{prefix}_flair_z{z}.png"
    plt.savefig(f1, dpi=150)
    plt.close()

    # Save overlay
    plt.figure(figsize=(4,4))
    plt.imshow(sl_img, cmap="gray")
    plt.imshow(np.ma.masked_where(~sl_mask, sl_mask), cmap="Reds", alpha=0.4)
    plt.axis("off")
    plt.title(f"Overlay (WT) z={z}")
    plt.tight_layout()
    f2 = out_dir / f"{prefix}_overlay_z{z}.png"
    plt.savefig(f2, dpi=150)
    plt.close()
    return f1, f2

In [ ]:
import os

current_directory = os.getcwd()
print("Current working directory:", current_directory)

In [ ]:
#testing the functions

case_dir = Path("data/BraTS2021_00495")  # one subject folder
modalities = ("flair",)                       # start with FLAIR only
min_pos = 200
out_dir = Path("outputs/step1")

vols, wt = load_case(case_dir, modalities=modalities)
flair = vols[modalities[0]]

H, W, D = flair.shape
print(f"Loaded: {case_dir.name}, shape H×W×D = {H}×{W}×{D}")
print(f"Total tumor voxels: {int(wt.sum())}")

z_list = pick_slices(wt, min_pos=min_pos)
if not z_list:
    print("No slices met min_pos. Try lowering min_pos.")
else:
    print(f"Found {len(z_list)} candidate slices (showing first 10):")
    print(z_list[:10])
    best_z, best_cnt = max(z_list, key=lambda t: t[1])
    print(f"Visualizing z={best_z} (tumor pixels={best_cnt})")
    f_flair, f_overlay = save_slice_and_overlay(flair, wt, best_z, out_dir, case_dir.name)
    print(f"Saved to:- {f_flair}- {f_overlay}")



NameError: name 'pwd' is not defined